In [1]:
using StaticArrays
using Parameters
using RobotDynamics
using LinearAlgebra

import RobotDynamics: dynamics
import RobotDynamics: state_dim, control_dim

In [2]:
@with_kw struct SphericalPendulum{T} <: AbstractModel
    m::T = 0.3
    length::T = 0.2
    g::T = 9.81
end

SphericalPendulum

In [15]:
function dynamics(p::SphericalPendulum, x, u)
    @unpack m, length, g = p
    θ = x[1]
    ϕ = x[2]
    θ̇ = x[3]
    ϕ̇ = x[4]
    
    θ̈ = ϕ̇^2*cos(θ)*sin(θ)-g/length*sin(θ)
    ϕ̈ = -2.0*ϕ̇*θ̇/tan(θ)
    @SVector [θ̇, ϕ̇, θ̈, ϕ̈]
end

dynamics (generic function with 9 methods)

In [4]:
state_dim(::SphericalPendulum) = 4
control_dim(::SphericalPendulum) = 0

control_dim (generic function with 4 methods)

In [5]:
using Colors
using CoordinateTransformations
using Rotations
using GeometryBasics
using MeshCat

┌ Info: Precompiling GeometryBasics [5c1252a2-5f33-56bf-86c9-59e7332b4326]
└ @ Base loading.jl:1278
┌ Info: Precompiling MeshCat [283c5d60-a78f-5afe-a0af-af636b173e11]
└ @ Base loading.jl:1278


In [6]:
vis = Visualizer()
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /Users/boom/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8700))

In [7]:
L = 1.0
p = SphericalPendulum(length = L)

kinematics(x) = (L*sin(x[1])*cos(x[2]), L*sin(x[1])*sin(x[2]), -L*cos(x[1]))

function visualize!(vis, model::SphericalPendulum, X, Δt)
    setobject!(vis[:ball], Sphere(Point3f0(0), 0.1),  
        MeshPhongMaterial(color = RGBA(0, 1, 0, 1.0)))

#     coordinates = [Point(.0, .0, .0), Point(.0, .0, -L)]
#     setobject!(vis[:line], Object(PointCloud(coordinates), 
#             LineBasicMaterial(color=RGBA{Float32}(1.0, 0.5, 0.14)), 
#             "Line"))
    pole = Cylinder(Point3f0(0,0,0),Point3f0(0,0,model.length),0.01f0)
    setobject!(vis[:pole], pole, MeshPhongMaterial(color=colorant"blue"))
    
    anim = MeshCat.Animation(convert(Int, floor(1.0 / Δt)))
    for (i, x) in enumerate(X)
        MeshCat.atframe(anim, i) do
            rot = LinearMap(RotZ(x[2])) ∘ LinearMap(RotY(π-x[1]))
            settransform!(vis[:ball], Translation(kinematics(x)...))
            settransform!(vis[:pole], rot)
            
#             coordinates = [Point(.0, .0, .0), Point(kinematics(x))]
#             setobject!(vis[:line], Object(PointCloud(coordinates), 
#             LineBasicMaterial(color=RGBA{Float32}(1.0, 0.5, 0.14)), 
#             "Line"))
        end
    end
    MeshCat.setanimation!(vis, anim)
end

visualize! (generic function with 1 method)

In [33]:
function dynamics_rk4(a::SphericalPendulum,x,u,h)
    f1 = dynamics(a, x, u)
    f2 = dynamics(a, x + 0.5*h*f1, u)
    f3 = dynamics(a, x + 0.5*h*f2, u)
    f4 = dynamics(a, x + h*f3, u)
    result = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    return result
end

dynamics_rk4 (generic function with 1 method)

In [36]:
tf = 10.0
Δt = 0.01
time = range(0, tf, step=Δt)
N = Int(round(tf/Δt)) + 1

init_pos = @SVector [π/10, π/10, 0.00001, 0.9]
X = [@SVector zeros(4) for k = 1:N] 
X[1] = init_pos

for k = 1:length(time) - 1
    try
        X[k+1] = dynamics_rk4(p,X[k],0,Δt)
    catch DomainError
        @show (X)
    end
end

visualize!(vis, p, X, Δt)